In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 

In [2]:
import torch
from torch import nn

from model.pspnet import PSPNet

In [20]:
# the origional model setting
criterion = nn.CrossEntropyLoss(ignore_index=255)
model = PSPNet(
    layers=50,
    classes=150,
    zoom_factor=8,
    pretrained=False,
    criterion=criterion,
)
model = torch.nn.DataParallel(model)
ckpt_pth = '/home/quanta/Models/PSPNet/pspnet_res50_train_epoch_100.pth'
state_dict = torch.load(ckpt_pth)['state_dict']
model.load_state_dict(state_dict)
# from DP to original model
model = model.module.cpu()

In [21]:
model

PSPNet(
  (criterion): CrossEntropyLoss()
  (layer0): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(128, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, ke

In [28]:
# nyu 40 have 40 class + void so classes = 41
model.cls[4] = nn.Conv2d(512, 41, kernel_size=1)
model.aux[4] = nn.Conv2d(256, 41, kernel_size=1)
model.criterion = nn.CrossEntropyLoss(ignore_index=0)

In [29]:
torch.save(model.state_dict(), '/home/quanta/Models/PSPNet/pspnet_res50_ade20k-init_scannet-adapt.pth')

In [30]:
# test if new model can be loaded
new_model = PSPNet(
    layers=50,
    classes=41,
    zoom_factor=8,
    pretrained=False,
    criterion=nn.CrossEntropyLoss(ignore_index=0),
)

In [31]:
new_model.load_state_dict(torch.load('/home/quanta/Models/PSPNet/pspnet_res50_ade20k-init_scannet-adapt.pth'))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])